In [2]:
import torch
import numpy as np
import argparse
import os,sys
import os.path as osp

from models import FlowNet2  # the path is depended on where you create this module
from utils.frame_utils import read_gen  # the path is depended on where you create this module
from scipy.misc import imresize

import cvbase as cvb
import matplotlib.pyplot as plt
# to visualize a flow file
# cvb.show_flow('output.flo')
from cvbase.image import rgb2bgr
from cvbase.visualize import show_img
from cvbase.optflow.io import read_flow
def flow2rgb(flow, color_wheel=None, unknown_thr=1e6):
    """Convert flow map to RGB image
    Args:
        flow(ndarray): optical flow
        color_wheel(ndarray or None): color wheel used to map flow field to RGB
            colorspace. Default color wheel will be used if not specified
        unknown_thr(str): values above this threshold will be marked as unknown
            and thus ignored
    
    Returns:
        ndarray: an RGB image that can be visualized
    """
    assert flow.ndim == 3 and flow.shape[-1] == 2
    if color_wheel is None:
        color_wheel = make_color_wheel()
    assert color_wheel.ndim == 2 and color_wheel.shape[1] == 3
    num_bins = color_wheel.shape[0]

    dx = flow[:, :, 0].copy()
    dy = flow[:, :, 1].copy()

    ignore_inds = (np.isnan(dx) | np.isnan(dy) | (np.abs(dx) > unknown_thr) |
                   (np.abs(dy) > unknown_thr))
    dx[ignore_inds] = 0
    dy[ignore_inds] = 0

    rad = np.sqrt(dx**2 + dy**2)
    if np.any(rad > np.finfo(float).eps):
        max_rad = np.max(rad)
        dx /= max_rad
        dy /= max_rad

    [h, w] = dx.shape

    rad = np.sqrt(dx**2 + dy**2)
    angle = np.arctan2(-dy, -dx) / np.pi

    bin_real = (angle + 1) / 2 * (num_bins - 1)
    bin_left = np.floor(bin_real).astype(int)
    bin_right = (bin_left + 1) % num_bins
    w = (bin_real - bin_left.astype(np.float32))[..., None]
    flow_img = (
        1 - w) * color_wheel[bin_left, :] + w * color_wheel[bin_right, :]
    small_ind = rad <= 1
    flow_img[small_ind] = 1 - rad[small_ind, None] * (1 - flow_img[small_ind])
    flow_img[np.logical_not(small_ind)] *= 0.75

    flow_img[ignore_inds, :] = 0

    return flow_img

def make_color_wheel(bins=None):
    """Build a color wheel
    Args:
        bins(list or tuple, optional): specify number of bins for each color
            range, corresponding to six ranges: red -> yellow, yellow -> green,
            green -> cyan, cyan -> blue, blue -> magenta, magenta -> red.
            [15, 6, 4, 11, 13, 6] is used for default (see Middlebury).
    
    Returns:
        ndarray: color wheel of shape (total_bins, 3)
    """
    if bins is None:
        bins = [15, 6, 4, 11, 13, 6]
    assert len(bins) == 6

    RY, YG, GC, CB, BM, MR = tuple(bins)

    ry = [1, np.arange(RY) / RY, 0]
    yg = [1 - np.arange(YG) / YG, 1, 0]
    gc = [0, 1, np.arange(GC) / GC]
    cb = [0, 1 - np.arange(CB) / CB, 1]
    bm = [np.arange(BM) / BM, 0, 1]
    mr = [1, 0, 1 - np.arange(MR) / MR]

    num_bins = RY + YG + GC + CB + BM + MR

    color_wheel = np.zeros((3, num_bins), dtype=np.float32)

    col = 0
    for i, color in enumerate([ry, yg, gc, cb, bm, mr]):
        for j in range(3):
            color_wheel[j, col:col + bins[i]] = color[j]
        col += bins[i]

    return color_wheel.T

    
    
    


In [11]:
# initial a Net
args = argparse.Namespace(fp16=False, rgb_max=255.0)
net = FlowNet2(args).cuda()
# load the state_dict
dict = torch.load("/playpen1/qiuyang/refined_segmentation/flownet2-pytorch/FlowNet2_checkpoint.pth.tar")
net.load_state_dict(dict["state_dict"])

Teli_5_rot_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_5_rot'
Teli_6_rot_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_6_rot'
flo_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_flo'
png_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_png'
npy_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_npy'

Teli_5_rot_img = osp.join(Teli_5_rot_path, 'F%05d_Teli_5.png')
Teli_6_rot_img = osp.join(Teli_6_rot_path, 'F%05d_Teli_6.png')
Teli_flo = osp.join(flo_path, 'F%05d_Teli.flo')
Teli_png = osp.join(png_path, 'F%05d_Teli.png')
Teli_npy = osp.join(npy_path, 'F%05d_Teli.npy')

for path in [flo_path,png_path,npy_path]:
    if not osp.isdir(path):
        os.makedirs(path)   

for fn in range(0, 628):
    pim1 = read_gen(Teli_5_rot_img%fn)
    pim2 = read_gen(Teli_6_rot_img%fn)
    # to the multiple of 64
#     h = (int(pim1.shape[0]/64)*64)
#     w = (int(pim1.shape[1]/64)*64)
    pim1 = imresize(pim1, (1216,1024))
    pim2 = imresize(pim2, (1216,1024))
    images = [pim1, pim2]
    images = np.array(images).transpose(3, 0, 1, 2)
    im = torch.from_numpy(images.astype(np.float32)).unsqueeze(0).cuda()
    
    # process the image pair to obtian the flow
    result = net(im).squeeze()
    # save flow, I reference the code in scripts/run-flownet.py in flownet2-caffe project
    def writeFlow(name, flow):
        f = open(name, 'wb')
        f.write('PIEH'.encode('utf-8'))
        np.array([flow.shape[1], flow.shape[0]], dtype=np.int32).tofile(f)
        flow = flow.astype(np.float32)
        flow.tofile(f)
        f.flush()
        f.close()
        
    data = result.data.cpu().numpy().transpose(1, 2, 0)
    # save as flo
    writeFlow(Teli_flo%fn, data)
    # save as npy
    np.save(Teli_npy%fn, data)
    # save as png
    flow = read_flow(data)
    flow_img = flow2rgb(flow)
    plt.imsave(Teli_png%fn, flow_img)
    

/playpen1/qiuyang/miniconda3/envs/flownet2-36/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/playpen1/qiuyang/miniconda3/envs/flownet2-36/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


In [3]:
# Oct 9 resized version
# initial a Net
args = argparse.Namespace(fp16=False, rgb_max=255.0)
net = FlowNet2(args).cuda()
# load the state_dict
dict = torch.load("/playpen1/qiuyang/refined_segmentation/flownet2-pytorch/FlowNet2_checkpoint.pth.tar")
net.load_state_dict(dict["state_dict"])

Teli_5_rot_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_5_rot'
Teli_6_rot_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_6_rot'
flo_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_flo_resized'
png_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_png_resized'
npy_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_npy_resized'

Teli_5_rot_img = osp.join(Teli_5_rot_path, 'F%05d_Teli_5.png')
Teli_6_rot_img = osp.join(Teli_6_rot_path, 'F%05d_Teli_6.png')
Teli_flo = osp.join(flo_path, 'F%05d_Teli.flo')
Teli_png = osp.join(png_path, 'F%05d_Teli.png')
Teli_npy = osp.join(npy_path, 'F%05d_Teli.npy')

for path in [flo_path,png_path,npy_path]:
    if not osp.isdir(path):
        os.makedirs(path)   

for fn in range(0, 628):
    pim1 = read_gen(Teli_5_rot_img%fn)
    pim2 = read_gen(Teli_6_rot_img%fn)
    # to the multiple of 64
#     h = (int(pim1.shape[0]/64)*64)
#     w = (int(pim1.shape[1]/64)*64)
    pim1 = imresize(pim1, (608,512))
    pim2 = imresize(pim2, (608,512))
    images = [pim1, pim2]
    images = np.array(images).transpose(3, 0, 1, 2)
    print(images.shape)
    im = torch.from_numpy(images.astype(np.float32)).unsqueeze(0).cuda()
    
    # process the image pair to obtian the flow
    result = net(im).squeeze()
    # save flow, I reference the code in scripts/run-flownet.py in flownet2-caffe project
    def writeFlow(name, flow):
        f = open(name, 'wb')
        f.write('PIEH'.encode('utf-8'))
        np.array([flow.shape[1], flow.shape[0]], dtype=np.int32).tofile(f)
        flow = flow.astype(np.float32)
        flow.tofile(f)
        f.flush()
        f.close()
        
    data = result.data.cpu().numpy().transpose(1, 2, 0)
    break
    # save as flo
    writeFlow(Teli_flo%fn, data)
    # save as npy
    np.save(Teli_npy%fn, data)
    # save as png
    flow = read_flow(data)
    flow_img = flow2rgb(flow)
    plt.imsave(Teli_png%fn, flow_img)
    

(3, 2, 608, 512)


/playpen1/qiuyang/miniconda3/envs/flownet2-36/lib/python3.6/site-packages/ipykernel_launcher.py:31: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/playpen1/qiuyang/miniconda3/envs/flownet2-36/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 1. Got 20 and 19 in dimension 2 at /opt/conda/conda-bld/pytorch_1544199946412/work/aten/src/THC/generic/THCTensorMath.cu:83

In [7]:
# Oct 13 version only Teli 5 frames
# initial a Net
args = argparse.Namespace(fp16=False, rgb_max=255.0)
net = FlowNet2(args).cuda()
# load the state_dict
dict = torch.load("/playpen1/qiuyang/refined_segmentation/flownet2-pytorch/FlowNet2_checkpoint.pth.tar")
net.load_state_dict(dict["state_dict"])

Teli_5_rot_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_5_rot'
flo_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_5_flo'
png_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_5_flo_png'
npy_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_5_flo_npy'

Teli_5_rot_img = osp.join(Teli_5_rot_path, 'F%05d_Teli_5.png')
Teli_flo = osp.join(flo_path, 'F%05d_Teli_5.flo')
Teli_png = osp.join(png_path, 'F%05d_Teli_5.png')
Teli_npy = osp.join(npy_path, 'F%05d_Teli_5.npy')

for path in [flo_path,png_path,npy_path]:
    if not osp.isdir(path):
        os.makedirs(path)   

for fn in range(0, 627):
    pim1 = read_gen(Teli_5_rot_img%fn)
    pim2 = read_gen(Teli_5_rot_img%(fn+1))
    # to the multiple of 64
#     h = (int(pim1.shape[0]/64)*64)
#     w = (int(pim1.shape[1]/64)*64)
    pim1 = imresize(pim1, (1216,1024))
    pim2 = imresize(pim2, (1216,1024))
    images = [pim1, pim2]
    
    images = np.array(images).transpose(3, 0, 1, 2)
    print(images.shape)
    im = torch.from_numpy(images.astype(np.float32)).unsqueeze(0).cuda()
    print(im.shape)
    # process the image pair to obtian the flow
    result = net(im).squeeze()
    # save flow, I reference the code in scripts/run-flownet.py in flownet2-caffe project
    def writeFlow(name, flow):
        f = open(name, 'wb')
        f.write('PIEH'.encode('utf-8'))
        np.array([flow.shape[1], flow.shape[0]], dtype=np.int32).tofile(f)
        flow = flow.astype(np.float32)
        flow.tofile(f)
        f.flush()
        f.close()
        
    data = result.data.cpu().numpy().transpose(1, 2, 0)
    # save as flo
    writeFlow(Teli_flo%fn, data)
    # save as npy
    np.save(Teli_npy%fn, data)
    # save as png
    flow = read_flow(data)
    flow_img = flow2rgb(flow)
    plt.imsave(Teli_png%fn, flow_img)

/playpen1/qiuyang/miniconda3/envs/flownet2-36/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/playpen1/qiuyang/miniconda3/envs/flownet2-36/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 

(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 

(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 

(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 

(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])
(3, 2, 1216, 1024)
torch.Size([1, 3, 2, 1216, 1024])


In [7]:
# Oct 14 FlownetCSS
# initial a Net
from models import FlowNet2CSS
args = argparse.Namespace(fp16=False, rgb_max=255.0)
net = FlowNet2CSS(args).cuda()
# load the state_dict
dict = torch.load("/playpen1/qiuyang/refined_segmentation/flownet2-pytorch/FlowNet2-CSS_checkpoint.pth.tar")
net.load_state_dict(dict["state_dict"])

Teli_5_rot_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_5_rot'
Teli_6_rot_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_6_rot'
flo_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Flownet2CSS_Teli_flo/'
png_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Flownet2CSS_Teli_png/'
npy_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Flownet2CSS_Teli_npy/'

Teli_5_rot_img = osp.join(Teli_5_rot_path, 'F%05d_Teli_5.png')
Teli_6_rot_img = osp.join(Teli_6_rot_path, 'F%05d_Teli_6.png')
Teli_flo = osp.join(flo_path, 'F%05d_Teli.flo')
Teli_png = osp.join(png_path, 'F%05d_Teli.png')
Teli_npy = osp.join(npy_path, 'F%05d_Teli.npy')

for path in [flo_path,png_path,npy_path]:
    if not osp.isdir(path):
        os.makedirs(path)   

for fn in range(0, 628):
    pim1 = read_gen(Teli_5_rot_img%fn)
    pim2 = read_gen(Teli_6_rot_img%fn)
    # to the multiple of 64
#     h = (int(pim1.shape[0]/64)*64)
#     w = (int(pim1.shape[1]/64)*64)
    pim1 = imresize(pim1, (1216,1024))
    pim2 = imresize(pim2, (1216,1024))
    images = [pim1, pim2]
    images = np.array(images).transpose(3, 0, 1, 2)
    im = torch.from_numpy(images.astype(np.float32)).unsqueeze(0).cuda()
    
    # process the image pair to obtian the flow
    result = net(im).squeeze()
    
    # save flow, I reference the code in scripts/run-flownet.py in flownet2-caffe project
    def writeFlow(name, flow):
        f = open(name, 'wb')
        f.write('PIEH'.encode('utf-8'))
        np.array([flow.shape[1], flow.shape[0]], dtype=np.int32).tofile(f)
        flow = flow.astype(np.float32)
        flow.tofile(f)
        f.flush()
        f.close()
        
    data = result.data.cpu().numpy().transpose(1, 2, 0)
    # save as flo
    writeFlow(Teli_flo%fn, data)
    # save as npy
    np.save(Teli_npy%fn, data)
    # save as png
    flow = read_flow(data)
    flow_img = flow2rgb(flow)
    plt.imsave(Teli_png%fn, flow_img)

/playpen1/qiuyang/miniconda3/envs/flownet2-36/lib/python3.6/site-packages/ipykernel_launcher.py:32: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
/playpen1/qiuyang/miniconda3/envs/flownet2-36/lib/python3.6/site-packages/ipykernel_launcher.py:33: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


In [ ]:
# Oct 17 FlownetCSS left-eye
# initial a Net
from models import FlowNet2CSS
args = argparse.Namespace(fp16=False, rgb_max=255.0)
net = FlowNet2CSS(args).cuda()
# load the state_dict
dict = torch.load("/playpen1/qiuyang/refined_segmentation/flownet2-pytorch/FlowNet2-CSS_checkpoint.pth.tar")
net.load_state_dict(dict["state_dict"])

Teli_5_rot_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Teli_5_rot'
flo_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Flownet2CSS_Teli_5_flo//'
png_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Flownet2CSS_Teli_5_png/'
npy_path = r'/playpen1/qiuyang/refined_segmentation/Oct_4_flownet2_and_stereo_data/Flownet2CSS_Teli_5_npy/'

Teli_5_rot_img = osp.join(Teli_5_rot_path, 'F%05d_Teli_5.png')
Teli_flo = osp.join(flo_path, 'F%05d_Teli.flo')
Teli_png = osp.join(png_path, 'F%05d_Teli.png')
Teli_npy = osp.join(npy_path, 'F%05d_Teli.npy')

for path in [flo_path,png_path,npy_path]:
    if not osp.isdir(path):
        os.makedirs(path)   

for fn in range(0, 627):
    pim1 = read_gen(Teli_5_rot_img%fn)
    pim2 = read_gen(Teli_5_rot_img%(fn+1))
    # to the multiple of 64
#     h = (int(pim1.shape[0]/64)*64)
#     w = (int(pim1.shape[1]/64)*64)
    pim1 = imresize(pim1, (1216,1024))
    pim2 = imresize(pim2, (1216,1024))
    images = [pim1, pim2]
    images = np.array(images).transpose(3, 0, 1, 2)
    im = torch.from_numpy(images.astype(np.float32)).unsqueeze(0).cuda()
    
    # process the image pair to obtian the flow
    result = net(im).squeeze()
    
    # save flow, I reference the code in scripts/run-flownet.py in flownet2-caffe project
    def writeFlow(name, flow):
        f = open(name, 'wb')
        f.write('PIEH'.encode('utf-8'))
        np.array([flow.shape[1], flow.shape[0]], dtype=np.int32).tofile(f)
        flow = flow.astype(np.float32)
        flow.tofile(f)
        f.flush()
        f.close()
        
    data = result.data.cpu().numpy().transpose(1, 2, 0)
    # save as flo
    writeFlow(Teli_flo%fn, data)
    # save as npy
    np.save(Teli_npy%fn, data)
    # save as png
    flow = read_flow(data)
    flow_img = flow2rgb(flow)
    plt.imsave(Teli_png%fn, flow_img)

/playpen1/qiuyang/refined_segmentation/flownet2-pytorch/models.py:460: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  init.uniform(m.bias)
/playpen1/qiuyang/refined_segmentation/flownet2-pytorch/models.py:461: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(m.weight)
/playpen1/qiuyang/refined_segmentation/flownet2-pytorch/models.py:465: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  init.uniform(m.bias)
/playpen1/qiuyang/refined_segmentation/flownet2-pytorch/models.py:466: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(m.weight)
/playpen1/qiuyang/miniconda3/envs/flownet2-36/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
    Use Pillow instead: ``numpy.array(Image.f